<a href="https://colab.research.google.com/github/nilabja-bhattacharya/WMT14_news_tranlation/blob/master/nmt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%matplotlib inline

In [3]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [0]:
path = "/gdrive/My Drive/WMT14_news_tranlation/nmt model/model/"

In [0]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.autograd import Variable
import torch.utils.data as Data
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
from nltk.translate import bleu
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction
plt.switch_backend('agg')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Language Model from input and output sentences

In [0]:
SOS_token = 0
EOS_token = 1
UNK_token = 2


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS", 2:"UNK"}
        self.n_words = 3  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [0]:
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
#     s = re.sub(r"([.!?|])", r" \1", s)
    #s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [0]:
  def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
#     lines = open('%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
#         read().strip().split('\n')
    lines1 = open("/gdrive/My Drive/corpus/training/hindien-train.tok."+lang1).readlines()
    lines2 = open("/gdrive/My Drive/corpus/training/hindien-train.tok."+lang2).readlines()
    #print(lines)
    # Split every line into pairs and normalize
    pairs = [[normalizeString(x), normalizeString(y)] for (x,y) in zip(lines1, lines2)]
#     pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]
#     print(pairs[:50])
    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [0]:
MAX_LENGTH = 25

def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [49]:
def prepareData(lang1, lang2, reverse=False):    
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    #print(input_lang, output_lang, pairs)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

input_lang, output_lang, pairs = prepareData('en', 'hi', True)
#print(input_lang, output_lang, pairs)
print(random.choice(pairs))

Reading lines...
Read 213492 sentence pairs
Trimmed to 153189 sentence pairs
Counting words...
Counted words:
hi 50584
en 51881
['कोई अलार  म सट नही ह', 'no alarm set']


## Encoder 

In [0]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.embedding = nn.Embedding(input_size, hidden_size)
        #changes
        self.gru = nn.LSTM(hidden_size, hidden_size, num_layers, bidirectional=True)

    def forward(self, input, hidden, cell):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, (hidden, cell) = self.gru(output, (hidden, cell))
        return output, hidden, cell

    def initHidden(self):
        return torch.zeros(self.num_layers*2, 1, self.hidden_size, device=device)

## Decoder

In [0]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, num_layers):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.embedding = nn.Embedding(output_size, hidden_size)
        #changes
        self.gru = nn.LSTM(hidden_size, hidden_size, num_layers, bidirectional=True)
        self.out = nn.Linear(hidden_size*2, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden, cell):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, (hidden, cell) = self.gru(output, (hidden, cell))
        output = self.softmax(self.out(output[0]))
        return output, hidden, cell

    def initHidden(self):
        return torch.zeros(self.num_layers*2, 1, self.hidden_size, device=device)

## Attention Based Decoder

In [0]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, num_layers,  dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length
        self.num_layers = num_layers
        self.embedding = nn.Embedding(self.output_size, self.hidden_size*2)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size *4 , self.hidden_size)
        self.attn_general = nn.Linear(self.max_length, self.max_length)
        self.attn_concat = nn.Linear(self.hidden_size*3, self.max_length)
        self.lcl_wa_into_hs = nn.Linear(self.hidden_size*2,self.hidden_size*2)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.LSTM(self.hidden_size, self.hidden_size, self.num_layers, bidirectional=True)
        self.out = nn.Linear(self.hidden_size*2, self.output_size)

    def forward(self, input, hidden, cell, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)
        #print(embedded.size(), hidden.size())
        # attn_weights = F.softmax(
        #     self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        X = None
        #print(embedded.shape, encoder_outputs.shape)
        if attention_type == 'dot':
            X = torch.matmul(embedded[0], encoder_outputs.T) #dot product
        elif attention_type == 'general':
            X = self.attn_general(torch.matmul(self.lcl_wa_into_hs(embedded[0]), encoder_outputs.T)) ## general
        elif attention_type == 'concat':
            X = self.attn_concat(torch.cat((embedded[0], hidden[0]), 1))  #concat
        attn_weights = F.softmax(X, dim = 1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))
        #print(attn_applied.shape)
        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, (hidden, cell) = self.gru(output, (hidden, cell))

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, cell, attn_weights

    def initHidden(self):
        return torch.zeros(self.num_layers*2, 1, self.hidden_size, device=device)

In [0]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] if word in lang.word2index.keys() else UNK_token for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    #print(indexes)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

## Training of Neural Network

In [0]:
teacher_forcing_ratio = 0.5
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()
    encoder_cell = encoder.initHidden()
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size*2, device=device)

    loss = 0

    for ei in range(input_length):
        #print(ei)
        encoder_output, encoder_hidden, encoder_cell = encoder(
            input_tensor[ei], encoder_hidden, encoder_cell)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden
    decoder_cell = encoder_cell
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            if attention == True:
                decoder_output, decoder_hidden, decoder_cell, decoder_attention = decoder(
                    decoder_input, decoder_hidden, decoder_cell, encoder_outputs)
            else:
                decoder_output, decoder_hidden, decoder_cell = decoder(
                    decoder_input, decoder_hidden, decoder_cell)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            if attention == True:
                decoder_output, decoder_hidden, decoder_cell, decoder_attention = decoder(
                    decoder_input, decoder_hidden, decoder_cell, encoder_outputs)
            else:
                decoder_output, decoder_hidden, decoder_cell = decoder(
                    decoder_input, decoder_hidden, decoder_cell)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [0]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

## Gradient Descent

In [0]:
 def trainIters(encoder, decoder, n_iters, print_every=1000, learning_rate=0.001):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every
    # lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False
    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=True)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate,  betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=True)
    if from_checkpoint == True:
      checkpoint_encoder = torch.load(path+"checkpoint.encoder_checkpoint.attn_concat."+str(itr)+".pth")
      encoder.load_state_dict(checkpoint_encoder['model_state_dict'])
      encoder_optimizer.load_state_dict(checkpoint_encoder['optimizer_state_dict'])
      checkpoint_decoder = torch.load(path+"checkpoint.decoder_checkpoint.attn_concat."+str(itr)+".pth")
      decoder.load_state_dict(checkpoint_decoder['model_state_dict'])
      decoder_optimizer.load_state_dict(checkpoint_decoder['optimizer_state_dict'])
      epoch = checkpoint_encoder['epoch']
    
    criterion = nn.NLLLoss()

    for iter in range(itr+1, itr + n_iters + 1):
        print("Epoch:{0}".format(iter))
        training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(len(pairs))]
        for b in range(1,len(pairs)+1):
              training_pair = training_pairs[b-1]
              input_tensor = training_pair[0]
              target_tensor = training_pair[1]

              loss = train(input_tensor, target_tensor, encoder,
                          decoder, encoder_optimizer, decoder_optimizer, criterion)
              print_loss_total += loss
              plot_loss_total += loss
        

        print_loss_avg = print_loss_total / len(pairs)
        print_loss_total = 0
        plot_losses.append(print_loss_avg)
        print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                      iter, iter / n_iters * 100, print_loss_avg))
        torch.save({
        'epoch': iter,
        'model_state_dict': encoder.state_dict(),
        'optimizer_state_dict': encoder_optimizer.state_dict(),
        }, path+"checkpoint.encoder_checkpoint.attn_concat10."+str(iter)+".pth")
        torch.save({
        'epoch': iter,
        'model_state_dict': decoder.state_dict(),
        'optimizer_state_dict': decoder_optimizer.state_dict(),
        }, path+"checkpoint.decoder_checkpoint.attn_concat10."+str(iter)+".pth")

    #showPlot(plot_losses)
    return plot_losses

In [0]:
def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

## Evaluation of test data

In [0]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()
        encoder_cell = encoder.initHidden()
        encoder_outputs = torch.zeros(max_length, encoder.hidden_size*2, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden, encoder_cell = encoder(input_tensor[ei],
                                                     encoder_hidden, encoder_cell)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden
        decoder_cell = encoder_cell

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            if attention == True:
                decoder_output, decoder_hidden, decoder_cell, decoder_attention = decoder(
                    decoder_input, decoder_hidden, decoder_cell, encoder_outputs)
                decoder_attentions[di] = decoder_attention.data
            else:
                decoder_output, decoder_hidden, decocer_cell = decoder(
                    decoder_input, decoder_hidden, decoder_cell)
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == UNK_token:
                decoded_words.append('<UNK>')
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [0]:
def calculate_bleu(pred_trg, real_trg):   
    smoothie = SmoothingFunction().method7
    return sentence_bleu(real_trg, pred_trg, smoothing_function=smoothie)
    # smoothie = SmoothingFunction().method4
    # return bleu(real_trg, pred_trg, smoothing_function=smoothie)
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')
        #print(calculate_bleu(pair[1], output_words[:-1]))

In [0]:
def showAttention(input_sentence, output_words, attentions):
    # Set up figure with colorbar
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(attentions.numpy()[:20][:20], cmap='bone')
    fig.colorbar(cax)

    # Set up axes
    ax.set_xticklabels([''] + input_sentence.split(' ') +
                       ['<EOS>'], rotation=90)
    ax.set_yticklabels([''] + output_words)

    # Show label at every tick
    ax.xaxis.set_major_locator(ticker.MultipleLocator(2))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(2))

    plt.show()

def evaluateAndShowAttention(encoder, decoder, input_sentence, real_output_sentence):
    output_words, attentions = evaluate(
        encoder, decoder, input_sentence)
    output_sentence = ' '.join(output_words)
    # print('input =', input_sentence)
    # print('output =', ' '.join(output_words))
    # print('>', input_sentence)
    # print('=', real_output_sentence)
    # print('<', output_sentence)
    # print('')
    blu = calculate_bleu(real_output_sentence, output_words[:-1])
    # if attention == True:
    #   showAttention(input_sentence, output_words, attentions)
    return blu


def evaluate_on_test(encoder, decoder):
  trg = open("/gdrive/My Drive/corpus/hindien-dev.tok.hi").readlines()
  real_out = open("/gdrive/My Drive/corpus/hindien-dev.tok.en").readlines()
  blu = 0
  cnt = 0
  for t, r in zip(trg, real_out):
      t = t.replace('\n','')
      r = r.replace('\n', '')
      try:
        if len(t.split(' ')) < MAX_LENGTH and len(r.split(' ')) < MAX_LENGTH:
            blu += evaluateAndShowAttention(encoder, decoder, t, r)
            cnt+=1
            if cnt == 2500:
              break
      except:
        pass
  print(cnt)
  return blu/cnt

## Implementation:

Here I'm implemention a sequence to sequence encoder decoder model of bi-LSTM with 256 hidden units using with and without attention and using various types of attention.

Encoder converts the input sequences to a normalized vector space and then decoder unit interpret the output sentence from this normalized vector space.

## Sequence to sequence model

In [0]:
from_checkpoint = True
save_checkpoint = True

In [0]:
hidden_size = 512
num_layers = 2
attention = False
itr = 5
torch.cuda.empty_cache() 
encoder1 = EncoderRNN(input_lang.n_words, hidden_size, num_layers).to(device)
decoder1 = DecoderRNN(hidden_size, output_lang.n_words, num_layers).to(device)
# losses = trainIters(encoder1, decoder1, 4)

In [0]:
#  showPlot(losses)

In [0]:
if from_checkpoint == True:
      checkpoint_encoder = torch.load(path+"checkpoint.encoder_checkpoint.seq."+str(itr)+".pth")
      encoder1.load_state_dict(checkpoint_encoder['model_state_dict'])
      checkpoint_decoder = torch.load(path+"checkpoint.decoder_checkpoint.seq."+str(itr)+".pth")
      decoder1.load_state_dict(checkpoint_decoder['model_state_dict'])

In [26]:
evaluateRandomly(encoder1, decoder1)
# print(blu)

> कटलन डस  कटॉप
= catalan desktop
< catalan desktop machine desktop desktop desktop desktop desktop desktop desktop desktop desktop desktop desktop desktop desktop desktop desktop desktop desktop desktop desktop desktop desktop desktop desktop desktop desktop desktop desktop desktop desktop desktop desktop desktop desktop desktop desktop desktop desktop desktop desktop desktop desktop desktop desktop desktop desktop desktop desktop desktop desktop desktop desktop desktop desktop desktop desktop desktop desktop desktop desktop desktop desktop desktop desktop desktop desktop desktop desktop desktop desktop desktop desktop desktop desktop desktop desktop desktop desktop desktop desktop desktop desktop desktop desktop desktop desktop desktop desktop desktop desktop desktop desktop desktop desktop desktop desktop desktop desktop

> लकिन य 400 साल परानी नाल , जिसम पानी बहता ह ,
= but this 400 year old canal , which draws water ,
< but it is this is this is this year this is this year this is

In [0]:
blu = evaluate_on_test(encoder1, decoder1)
print(blu)

## Attention Based Models

### DOT variant

In [0]:
hidden_size = 256
num_layers = 2
attention = True
itr = 10
attention_type = 'dot'
encoder1 = EncoderRNN(input_lang.n_words, hidden_size, num_layers).to(device)
decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, num_layers, dropout_p=0.1).to(device)
#losses = trainIters(encoder1, decoder1, 2, print_every=1)

In [0]:
#  showPlot(losses)

In [0]:
if from_checkpoint == True:
      checkpoint_encoder = torch.load(path+"checkpoint.encoder_checkpoint.attn_dot."+str(itr)+".pth")
      encoder1.load_state_dict(checkpoint_encoder['model_state_dict'])
      checkpoint_decoder = torch.load(path+"checkpoint.decoder_checkpoint.attn_dot."+str(itr)+".pth")
      decoder1.load_state_dict(checkpoint_decoder['model_state_dict'])

In [30]:
evaluateRandomly(encoder1, decoder1)

> ( बी ) प  रशिक  षण और कार  य अनभव
= -lrb- b -rrb- training and work experience
< ( applause -rrb- training and training training <EOS>

> और म अरब-इजरायल सघर  ष क सम  बन  ध म हमास की विजय को लकर तटस  थ ह .
= not much separates hamas anti-zionism from fatah anti-zionism except that hamas terrorists speak forthrightly while fatah terrorists obfuscate . even their tactics overlap , as fatah denies the existence of israel and hamas negotiates with israelis . differing emphases and styles , more than substance , distinguishes their attitudes toward israel .
< and i hamas hamas hamas anti-zionism anti-zionism the hamas the hamas the . . . . <EOS>

> सप  टिसीमिया क  या ह ?
= what is septicaemia ?
< what is the ? ? <EOS>

> स  वर
= vowel
< swara <EOS>

> पासवर  ड रख ( ओई सफ )
= set password ( oi safe )
< encrypt password ( oi safe ) <EOS>

> पर  चो म मसलमानो को नौकरी पर रखन और उनस खरीद-फरोत करन क खिलफ चतावनी दी गई ह .
= they warn against employing muslims , buying from them or selling to the

In [0]:
blu = evaluate_on_test(encoder1, decoder1)
print(blu)

### General Variant

#### Bi-LSTM with 256 unit and 2 Layers

In [0]:
hidden_size = 256
num_layers = 2
attention = True
itr = 9
attention_type = 'general'
encoder1 = EncoderRNN(input_lang.n_words, hidden_size, num_layers).to(device)
decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, num_layers, dropout_p=0.1).to(device)
#losses = trainIters(encoder1, decoder1, 100000, print_every=10)

In [0]:
if from_checkpoint == True:
      checkpoint_encoder = torch.load(path+"checkpoint.encoder_checkpoint.attn_general."+str(itr)+".pth")
      encoder1.load_state_dict(checkpoint_encoder['model_state_dict'])
      checkpoint_decoder = torch.load(path+"checkpoint.decoder_checkpoint.attn_general."+str(itr)+".pth")
      decoder1.load_state_dict(checkpoint_decoder['model_state_dict'])

In [0]:
#  showPlot(losses)

In [34]:
evaluateRandomly(encoder1, decoder1)

> उस गति स सबधित मीट  रिक जिनस % { short _ product _ name } अनरोधित कार  यवाही निष  पादित करता ह
= metrics relating to the speed with which % { short _ product _ name } performs requested actions
< metrics relating to add % { short _ frame _ frame } from the requested _ frame <EOS>

> उनकी निष   ठा अपन घरो म भ  ट  टियो म आग जलान , अपन दवता क लिए भजन गान और चावल , दध सोमरस या पश बलि क रप म अर  पित करन म थी .
= their devotion consisted in burning fires in their hearths , singing hymns to their gods and offering rice , milk , soma or animals as sacrifice .
< his speeches in his his , his his his , his , his , his , his , , and and and and and his , <EOS>

> बाबर और उसक पत  र हमाय दोनो क पास भारत म राजनतिक और सास   कतिक एकता उत   पन   न करन का महान कार  य करन की व   यापक दष   टि तथा कल   पना थी , कित उनका शासन बहत कम समय तक रहा .
= both babur and his son humayun had the breadth of vision and the imagination to set about the great task of creating political and cultural unity in india ; but

In [0]:
blu = evaluate_on_test(encoder1, decoder1)
print(blu)

#### Bi-LSTM with 512 unit and 4 layers

In [0]:
hidden_size = 512
num_layers = 4
attention = True
itr = 6
attention_type = 'general'
encoder1 = EncoderRNN(input_lang.n_words, hidden_size, num_layers).to(device)
decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, num_layers, dropout_p=0.1).to(device)
#losses = trainIters(encoder1, decoder1, 100000, print_every=10)

In [0]:
if from_checkpoint == True:
      checkpoint_encoder = torch.load(path+"checkpoint.encoder_checkpoint.attn_general512."+str(itr)+".pth")
      encoder1.load_state_dict(checkpoint_encoder['model_state_dict'])
      checkpoint_decoder = torch.load(path+"checkpoint.decoder_checkpoint.attn_general512."+str(itr)+".pth")
      decoder1.load_state_dict(checkpoint_decoder['model_state_dict'])

In [0]:
#  showPlot(losses)

In [38]:
evaluateRandomly(encoder1, decoder1)

> व टस  ट क  रिकट म सबस ज   यादा रन बनान वाल बल  लबाज  ह ।
= he has scored the maximum runs in test cricket .
< they is the the of the the cricket of . . <EOS>

> pulse aqui para cambiar su apodo
= click to change your nickname
< click to change the nickname <EOS>

> लवासा
= lavasa
< shirdi <EOS>

> changer de mot de passe ...
= change password ...
< change nickname ... <EOS>

> जाचसची
= checklist
< polish <EOS>

> म अपन प  रदाता नही ढढ  सकता ह और म इस दस  ती रप स दाखिल करना चाहता ह ( m ) :
= i can &apos;t find my provider and i wish to enter it manually :
< i can want to want to my my i i i i : <EOS>

> यह सब फ  रिश  त नही ह ,
= all of them are not angels ,
< the all is not not , <EOS>

> सशन कार  यक  रम
= session programs
< marquee survey <EOS>

> आप क  या कर सकत ह अगर आप को शिकायत करनी हो ।
= what to do if you wish to complain
< what if if if if if if you can . <EOS>

> सिर  फ बड अक  षरो का ही साराश लिख
= show uppercase text only
< only the the the the <EOS>



In [47]:
blu = evaluate_on_test(encoder1, decoder1)
print(blu)

2500
0.23886061729751382


#### Sentence length 25

In [0]:
hidden_size = 512
num_layers = 4
attention = True
itr = 12
attention_type = 'general'
encoder1 = EncoderRNN(input_lang.n_words, hidden_size, num_layers).to(device)
decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, num_layers, dropout_p=0.1).to(device)

In [0]:
if from_checkpoint == True:
      checkpoint_encoder = torch.load(path+"checkpoint.encoder_checkpoint.attn_general51225."+str(itr)+".pth")
      encoder1.load_state_dict(checkpoint_encoder['model_state_dict'])
      checkpoint_decoder = torch.load(path+"checkpoint.decoder_checkpoint.attn_general51225."+str(itr)+".pth")
      decoder1.load_state_dict(checkpoint_decoder['model_state_dict'])

In [0]:
#  showPlot(losses)

In [64]:
evaluateRandomly(encoder1, decoder1)

> एवोल  यशन बकअप जाच
= check evolution back up
< check evolution address <EOS>

> डिवाइस चयन
= device selection
< the device <EOS>

> नोबल परस  कार जीतन क बाद , वह चार दशको तक गमनामी म जीवन व  यतीत करता रहा ।
= after winning the noble prize , he lived in obscurity for four decades .
< after the father of his noble of the the he the father of his father . <EOS>

> घ  l18 आप को सामाजिक निधी स मदद
= gl18 help from the social fund
< gl18 you tenant you benefit <EOS>

> इडफिनिट
= indefinite
< rwanda <EOS>

> अल  लाह उस गमराह कर दगा ।
= allah will destroy them
< allah will make it misleading . <EOS>

> उन  होन पहल ही हम चतावनी दी थी । &lt; s &gt; यह शरआत ह ।
= they told us what to expect . now it ’ s beginning .
< they asked that we lesson . <EOS>

> तो हम एक दसरी धारणा की और कदम बढ  ात ह ,
= then we move on to a second assumption ,
< so we have a a a a a , , <EOS>

> क  यकि और कही ,
= because around the corner ,
< because that &apos;s , <EOS>

> कार  यक  रम का नाम बदल ( n ) ...
= rename eve

In [65]:
blu = evaluate_on_test(encoder1, decoder1)
print(blu)

2500
0.2752689768443108


### Concat Varient

In [0]:
hidden_size = 256
num_layers = 2
attention_type = 'concat'
attention = True
itr = 5
encoder1 = EncoderRNN(input_lang.n_words, hidden_size, num_layers).to(device)
decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, num_layers, dropout_p=0.1).to(device)
#losses = trainIters(encoder1, decoder1, 5, print_every=10)

In [0]:
if from_checkpoint == True:
      checkpoint_encoder = torch.load(path+"checkpoint.encoder_checkpoint.attn_concat."+str(itr)+".pth")
      encoder1.load_state_dict(checkpoint_encoder['model_state_dict'])
      checkpoint_decoder = torch.load(path+"checkpoint.decoder_checkpoint.attn_concat."+str(itr)+".pth")
      decoder1.load_state_dict(checkpoint_decoder['model_state_dict'])

In [0]:
#  showPlot(losses)

In [0]:
# torch.save(losses, 'losses_concat.pth')

In [43]:
evaluateRandomly(encoder1, decoder1)

> पिछल हत जब एअर इडिया का विशष विमान कष  णदव राय , जिस पर 16वी सदी क सम  राट का नाम लिखा हा था , प  रधानमत  री वाजपयी को विएतनाम और इडोनशिया ल गया तो भारत की नई पर  वाभिमख कटनीति म नया मोड आ गया .
= last week , when air-india &apos; s special aircraft , krishna deva raya , flew to vietnam and indonesia with prime minister atal bihari vajpayee , the name of the 16th century king painted on it added an ironic twist to india &apos; s new look-east diplomacy .
< in punjab , the punjab of punjab , punjab , punjab , punjab , punjab , punjab , punjab , punjab , punjab , punjab , punjab , the punjab of the punjab . the punjab of the punjab . <EOS>

> जिस जररत ह उसक लिय , एन एच एस व  दारा हियरिग एडज , ( श  रवण सहायक उपकरण ) बटरिया और उनकी मरम  मत मफ  त ह ।
= for those who need them , hearing aids batteries or repairs are free on the nhs .
< for example , for example , solvents , , , , , , , , , , , , , <EOS>

> श  री कष  ण न गीता का सन  दश अर  जन को सनाया था ।
= shree krishna told arjun about p

In [0]:
blu = evaluate_on_test(encoder1, decoder1)
print(blu)